In [97]:
#起始標準起手式~
import requests as rq
from bs4 import BeautifulSoup as bs
from collections import OrderedDict as od
import json
import csv
import traceback as tb
import re

In [98]:
#def dodocook_crawler(開始文章ID,結束文章ID)
def dodocook_crawler(no_start_page,no_stop_page):
    HOST = 'http://www.dodocook.com/recipe/'
    #在pass這裡接上function() 網址產生器
    #pass
    url_builder(HOST,no_start_page,no_stop_page)

In [99]:
#網址產生器
#url_builder(網站URL, 開始文章ID, 結束文章ID)
def url_builder(HOST,no_start_page,no_stop_page):
    for i in range(no_start_page, no_stop_page):
        url = HOST + "{}/".format(i)        
        print("[INFO] {}".format(url))
        #在pass這裡接上function()  res_and_soup(url)
        #pass
        res_and_soup(url)

In [117]:
#檢查食譜頁面是否存在。存在則進行食譜頁面爬蟲，被導回首頁則出現警告
def if_page_exist(url, soup):
    try:
        #pass
        #還沒有想到更好的檢查方式，目前使用檢查標題是否有"【多多開伙】"，有則表示回到首頁，會出現警告
        pattern = re.compile("【多多開伙】")
        match = re.search(pattern, soup.select("title")[0].text)
        
        #*****************************************
        #應該增加存放原始URL的DICT檔案，暫時懶得寫~~~
        #*****************************************
        #食譜頁面不存在，警告
        if match:
            print("[WARNING]Page redirects")
        
        #食譜頁面存在，使用食譜頁面爬蟲
        else:
            dodocook_contents(soup)    
    except:
        tb.print_exc()

In [113]:
#就字面意思
def res_and_soup(url):
    try:
        res = rq.get(url)
        print("[INFO] requests_success!")
        soup = bs(res.text, 'lxml')        
        #在pass這裡接上爬取網頁內容的function()
        #pass
        print("[INFO] soup_success!")
        
        #增加確認食譜頁面存在而非重新導向到首頁(2017/02/15)
        #dodocook_contents(soup)
        if_page_exist(url, soup)
        
        print("[INFO] completion!")
    except:
        print("[Error] Error while getting contents")
        tb.print_exc()

In [122]:
def dodocook_contents(soup):
    #為了排版整齊，使用OrderedDict來裝內容
    mydict = od()
    #title
    #2017/0214:新增try except來處理有時候抓取title碰到抓到標題卻出現list index out of range
    #修正方式暫時不明
    try:
        #title並非直接從<title>取得，因為<title>內容格式如下
        #ex:<title>腐乳脆花腩食譜、作法 | Mia的多多開伙食譜分享</title>
        title = soup.select(".band > h1")[0].text
        mydict["title"] = title
        
    #*****************************************    
    #不明原因的list index out of range，需要修正
    #*****************************************
    except IndexError:
        print("[Error] list index out of range")
    
    
    
    #Ingredients
    #***************************************** 
    #食材內容需要注意空值與正規化問題，還需要另外處理
    #***************************************** 
    Ingredients = []
    for tag2 in soup.select(".body > .list > .name"):
        tag2_dict={}
        n = soup.select(".body > .list > .name").index(tag2)
        tag2_dict[tag2.text] = soup.select(".ingredient > .body > .list > .count")[n].text
        Ingredients.append(tag2_dict)
    mydict["Ingredients"] = Ingredients
    
    #step(烹飪步驟)
    #dodocook目前有3種呈現步驟的格式，所以有三種選取方式
    #A模式
    if (len(soup.select('.steps > .CSrtA > .SrtAk > .Sno')) != 0):        
        
        
        #取步驟Index
        step_list_index = soup.select('.steps > .CSrtA > .SrtAk > .Sno') 
        #取步驟內容
        step_list_content = soup.select('.steps > .CSrtA > .SrtAk > .Spma')
        

        step_list=[]
        for i,step in enumerate(step_list_index):
            my_dict = {}
            #每個步驟的NO
            step_index = int(step.text)
            #print(step_index)
            my_dict["step_number"] = step_index
            #每個步驟的內容
            contents = step_list_content[i].text.strip()
            #print(contents)
            my_dict["step_content"] = contents
            step_list.append(my_dict)  
            
        #print(step_list)
        mydict["step"] = step_list
        
    #B模式
    elif (len(soup.select('.steps > .CSrtB > .Sno')) != 0):        
        #取步驟Index
        step_list_index = soup.select('.steps > .CSrtB > .Sno') 
        #取步驟內容
        step_list_content = soup.select('.steps > .CSrtB > .SBpma')
        

        step_list=[]
        for i,step in enumerate(step_list_index):
            my_dict = {}
            #每個步驟的NO
            step_index = int(step.text)
            #print(step_index)
            my_dict["step_number"] = step_index
            #每個步驟的內容
            contents = step_list_content[i].text.strip()
            #print(contents)
            my_dict["step_content"] = contents
            step_list.append(my_dict)  
            
        #print(step_list)
        mydict["step"] = step_list    
    
    #C模式
    else:        
        #取步驟Index
        step_list_index = soup.select('.steps > .CSrtC > .Sno')
        #取步驟內容
        step_list_content = soup.select('.steps > .CSrtC > .SCpma')
        

        step_list=[]
        for i,step in enumerate(step_list_index):
            my_dict = {}
            #每個步驟的Number
            step_index = int(step.text)
            #print(step_index)
            my_dict["step_number"] = step_index
            #每個步驟的內容
            contents = step_list_content[i].text.strip()
            #print(contents)
            my_dict["step_content"] = contents
            step_list.append(my_dict)  
            
        #print(step_list)
        mydict["step"] = step_list
    
    #最後確認步驟是否為空字典，如果是空字典則出現警告，要另外處理
    if not mydict["step"]:
        print("[***WARNING***]step's value is null")
    print(mydict)

In [123]:
dodocook_crawler(70000, 70100)

[INFO] http://www.dodocook.com/recipe/70000/
[INFO] requests_success!
[INFO] soup_success!
[WARNING]Page redirects
[INFO] completion!
[INFO] http://www.dodocook.com/recipe/70001/
[INFO] requests_success!
[INFO] soup_success!
OrderedDict([('title', '檸檬烤雞野菇燉飯'), ('Ingredients', [{'烤雞': '適量'}, {'生米': '1杯'}, {'蒜頭': '野菇'}, {'雞湯塊': '1塊'}]), ('step', [{'step_content': '熱油爆香蒜頭後炒生米', 'step_number': 1}, {'step_content': '加入烤雞絲及略淹過米的水量 大火煮', 'step_number': 2}, {'step_content': '反覆收乾再加水的過程3次 直到喜歡的米心熟度 我吃8分熟 略有米心 就可以加入甜椒辣椒起司野菇', 'step_number': 3}, {'step_content': '最後一次加水只需一點點 煮軟食材們即可囉 最後擠上檸檬汁', 'step_number': 4}])])
[INFO] completion!
[INFO] http://www.dodocook.com/recipe/70002/
[INFO] requests_success!
[INFO] soup_success!
OrderedDict([('title', '日式醬燒帆立貝'), ('Ingredients', [{'日本帆立貝': '6顆'}, {'綠花椰菜': '適量'}, {'醬汁:': ''}, {'醬油': '3大匙'}, {'味霖': '3大匙'}, {'紹興酒': '3大匙'}, {'砂糖': '2大匙'}]), ('step', [{'step_content': '冷藏解凍帆立貝略洗,綠花椰菜洗淨備用..', 'step_number': 1}, {'step_content': '水滾後,加入少許塩放入綠花椰燙熟即可撈起備用..', 's

In [104]:
testurl = 'http://www.dodocook.com/recipe/50001/'

In [105]:
res = rq.get(testurl)
soup = bs(res.text, 'lxml')

In [107]:
soup.select("title")[0].text

'食譜烹飪【多多開伙】第一棒廚藝社群網站'

In [106]:
pattern = re.compile("多多開伙")
match = re.search(pattern, soup.select("title")[0].text)
if match:
    print('true')
else:
    print('false')

true


In [75]:
result

<_sre.SRE_Match object; span=(5, 9), match='多多開伙'>

In [87]:
dir(Match)

NameError: name 'Match' is not defined

In [53]:
soup

<!DOCTYPE html>
<html class="no-js" lang="">
<head>
<meta charset="utf-8"/>
<link href="/images/favicon.ico" rel="shortcut icon"/> <title>食譜烹飪【多多開伙】第一棒廚藝社群網站</title>
<meta content="找食譜、紀錄食譜快來【多多開伙】，這邊有許多愛好廚藝、烹飪的同好們，一起分享您的食譜，紀錄烹飪生活的每一天！" name="description"/>
<meta content="食譜, 廚藝, 烹飪, 多多開伙, dodocook" name="keywords"/>
<meta content="http://www.dodocook.com/images/welcome.jpg" property="og:image"/>
<link href="/css/normalize.min.css" rel="stylesheet"/>
<link href="/css/1280/main.css" rel="stylesheet" type="text/css"/>
<link href="/css/1280/0625.css" rel="stylesheet" type="text/css"/>
<link href="/css/1024/1024.css" media="screen and (max-width: 1279px)" rel="stylesheet" type="text/css"/>
<link href="/css/collect_ajax.css" rel="stylesheet" type="text/css"/>
<link href="/js/fancybox/source/jquery.fancybox.css" rel="stylesheet" type="text/css"/> <link href="/css/pagebar.css" rel="stylesheet" type="text/css"/>
<script src="/js/vendor/modernizr-2.8.3.min.js"></script>
<script src="/js/jquery/